In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
import numpy as np
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Activation
from keras.callbacks import EarlyStopping
import h5py

#if u dont have Gpu, change the module name from "tensorflow-gpu" -> "tensorflow" in the file "virtual_platform_windows.yml" and
#run this command in conda terminal - conda env update -f virtual_platform_windows.yml

Using TensorFlow backend.


In [2]:
classifier = Sequential()

## Neural Network starts here

In [3]:
classifier.add(Convolution2D(32,(7,7),input_shape = (100, 100, 1),strides=(3,3),padding='valid'))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

In [4]:
classifier.add(Convolution2D(64,(5,5),strides=(2,2),padding='valid'))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))

In [5]:
classifier.add(Convolution2D(150,(3,3),strides=(1,1),padding='valid'))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

In [6]:
classifier.add(Flatten())

In [7]:
classifier.add(Dense(units=32))

In [8]:
from keras.layers import Dropout

In [9]:
classifier.add(Dropout(0.6))

In [10]:
classifier.add(Dense(units=25))
classifier.add(Dropout(0.6))

In [11]:
classifier.add(Dense(units=7,activation='softmax'))

## Neural Network ends here

In [12]:
sgd = SGD(lr=0.005, momentum=0.9)
classifier.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=5,
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

In [13]:
from keras.preprocessing.image  import ImageDataGenerator

In [14]:
train_datagen = ImageDataGenerator()

In [15]:
test_datagen = ImageDataGenerator()

In [16]:
training_set = train_datagen.flow_from_directory( 'D:/UTD_SEM/Fall_17/DIP/Project/ProjectDataset2/Train',
                                                    target_size=(100, 100),
                                                    color_mode="grayscale",
                                                    batch_size=128,
                                                    class_mode='categorical')

Found 56000 images belonging to 7 classes.


In [17]:
testing_set = test_datagen.flow_from_directory( 'D:/UTD_SEM/Fall_17/DIP/Project/ProjectDataset2/Test',
                                                    target_size=(100, 100),
                                                    color_mode="grayscale",
                                                    batch_size=128,
                                                    class_mode='categorical')


Found 280 images belonging to 7 classes.


In [18]:
classifier.fit_generator(training_set,
                            steps_per_epoch=14400*4/128,
                            epochs=8,
                            validation_data=testing_set,
                            callbacks=callbacks_list,
                            validation_steps=8000*4/128,workers=128,max_q_size=64)

C:\Anaconda3\envs\virtual_platform\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=450.0, callbacks=[<keras.ca..., max_queue_size=64, workers=128, epochs=8, validation_steps=250.0)`
  


Epoch 1/8
450/450 [==============================] - 68s 151ms/step - loss: 1.4262 - acc: 0.5431 - val_loss: 0.8188 - val_acc: 0.7109
Epoch 2/8
450/450 [==============================] - 63s 140ms/step - loss: 0.3428 - acc: 0.8789 - val_loss: 0.3703 - val_acc: 0.9143
Epoch 3/8
450/450 [==============================] - 63s 140ms/step - loss: 0.1903 - acc: 0.9335 - val_loss: 0.2386 - val_acc: 0.9321
Epoch 4/8
450/450 [==============================] - 62s 138ms/step - loss: 0.1350 - acc: 0.9523 - val_loss: 0.1930 - val_acc: 0.9499
Epoch 5/8
450/450 [==============================] - 62s 138ms/step - loss: 0.1016 - acc: 0.9641 - val_loss: 0.2280 - val_acc: 0.9537
Epoch 6/8
450/450 [==============================] - 64s 141ms/step - loss: 0.0826 - acc: 0.9694 - val_loss: 0.2302 - val_acc: 0.9643
Epoch 7/8
450/450 [==============================] - 63s 140ms/step - loss: 0.0718 - acc: 0.9737 - val_loss: 0.2943 - val_acc: 0.9572
Epoch 8/8
450/450 [==============================] - 63s 139ms

In [19]:
classifier.save("model")

In [20]:
from keras.preprocessing import image
y_pred = []

In [21]:
for j in range(7):
    for i in range(40):
        test_image = image.load_img('D:/UTD_SEM/Fall_17/DIP/Project/ProjectDataset2/Test/a'+str(1+j)+'/a'+str(1+j)+'_'+str(i+1)+'.png',target_size=(100, 100),grayscale=True)
        test_image = image.img_to_array(test_image)
        test_image.shape
        test_image = np.expand_dims(test_image,axis=0)
        result = classifier.predict(test_image)
        y_pred.append(result.argmax())

In [22]:
test_prediction = np.array(y_pred)

In [23]:
test_prediction.shape

(280,)

In [24]:
testing_set.classes.shape

(280,)

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(testing_set.classes, test_prediction)

array([[34,  0,  0,  4,  1,  1,  0],
       [ 0, 37,  2,  0,  1,  0,  0],
       [ 0,  0, 37,  0,  0,  3,  0],
       [ 0,  0,  0, 39,  0,  1,  0],
       [ 0,  1,  0,  0, 39,  0,  0],
       [ 0,  0,  0,  1,  0, 39,  0],
       [ 0,  0,  0,  0,  0,  0, 40]], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(testing_set.classes, test_prediction)*100

94.642857142857139

# Pretrained model goes here

In [29]:
from keras.models import load_model

In [33]:
classifier = load_model("model")